## Import des bibliothéques

In [1]:
import matplotlib.pyplot as plt
from scipy import signal
import tensorflow as tf
from tensorflow import keras
from data_transfo import get_train_test_fitted
import pandas as pd

2022-06-06 10:48:02.522427: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-06 10:48:02.522453: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Traitement des données

### Téléchargement des datasets

In [2]:
train_df = pd.read_csv('https://maxime-devanne.com/datasets/ECG200/ECG200_TRAIN.tsv', sep='\t', header=None)
test_df = pd.read_csv('https://maxime-devanne.com/datasets/ECG200/ECG200_TEST.tsv', sep='\t', header=None)

### Préparation des données de d'entrainement et de test

In [3]:
x_train, y_train, x_test, y_test = get_train_test_fitted(
                                                        train_df,
                                                        test_df
                                                        )

(200, 96)
(200, 96, 1)


## CNN

### Construction des layers

In [8]:
input_layer = keras.layers.Input(batch_shape=[None, x_train.shape[1], x_train.shape[2]])

lstm_1 = keras.layers.LSTM(units=15, stateful=False)(input_layer)

output_layer = keras.layers.Dense(activation='softmax',units=y_train.shape[1])(lstm_1)

model=keras.models.Model(inputs=input_layer, outputs=output_layer)
model.summary()

2022-06-06 10:51:15.153091: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-06 10:51:15.153134: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-06 10:51:15.153161: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cc-t14): /proc/driver/nvidia/version does not exist
2022-06-06 10:51:15.155073: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 1)]           0         
                                                                 
 lstm (LSTM)                 (None, 15)                1020      
                                                                 
 dense (Dense)               (None, 2)                 32        
                                                                 
Total params: 1,052
Trainable params: 1,052
Non-trainable params: 0
_________________________________________________________________


### Compilation du modèle

In [9]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])

### Paramétrage de l'entrainement

In [10]:
batch_size=20
nb_epoch = 250

### Entrainement

In [11]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=nb_epoch) #, verbose=0)
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/250
5/5 [==============================] - 1s 25ms/step - loss: 0.6862 - accuracy: 0.6500
Epoch 2/250
5/5 [==============================] - 0s 22ms/step - loss: 0.6808 - accuracy: 0.7400
Epoch 3/250
5/5 [==============================] - 0s 24ms/step - loss: 0.6748 - accuracy: 0.8000
Epoch 4/250
5/5 [==============================] - 0s 22ms/step - loss: 0.6685 - accuracy: 0.8100
Epoch 5/250
5/5 [==============================] - 0s 24ms/step - loss: 0.6607 - accuracy: 0.7800
Epoch 6/250
5/5 [==============================] - 0s 23ms/step - loss: 0.6522 - accuracy: 0.7900
Epoch 7/250
5/5 [==============================] - 0s 24ms/step - loss: 0.6413 - accuracy: 0.7400
Epoch 8/250
5/5 [==============================] - 0s 23ms/step - loss: 0.6294 - accuracy: 0.7100
Epoch 9/250
5/5 [==============================] - 0s 24ms/step - loss: 0.6104 - accuracy: 0.7100
Epoch 10/250
5/5 [==============================] - 0s 21ms/step - loss: 0.5906 - accuracy: 0.7000
Epoch 11/250
5/5 [=

### Evaluation sur les données de test

In [12]:
loss,acc = model.evaluate(x_test,y_test,verbose=False)

print("L'accuracy sur l'ensemble du test est:",acc)

L'accuracy sur l'ensemble du test est: 0.7300000190734863
